#Exploring the BigQuery API with ChEMBL

First, you'll need a Google Cloud API key. Download the JSON file to use the client.

[A guide is provided here](https://cloud.google.com/docs/authentication/getting-started)

In [ ]:
from google.cloud import bigquery
import os

api_path = '/content/drive/MyDrive/Colab Notebooks/chembl-api.json' #wherever you saved the key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_path

In [ ]:
client = bigquery.Client()

A dataset reference points BiqQuery to the correct dataset and project.

[View the database on Google Cloud](https://console.cloud.google.com/bigquery?project=patents-public-data&d=ebi_chembl&p=patents-public-data&page=dataset&pli=1)

In [ ]:
dset_ref = client.dataset('ebi_chembl', project='patents-public-data')
chembl_dset = client.get_dataset(dset_ref)

In [ ]:
tables = [x.table_id for x in client.list_tables(chembl_dset)]
print(len(tables), 'tables')
tables

In [ ]:
#29 indicates ChEMBL 29, the latest version
products = client.get_table(chembl_dset.table('products_29'))
products.schema

[SchemaField('dosage_form', 'STRING', 'NULLABLE', None, ()),
 SchemaField('route', 'STRING', 'NULLABLE', None, ()),
 SchemaField('trade_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('approval_date', 'STRING', 'NULLABLE', None, ()),
 SchemaField('ad_type', 'STRING', 'NULLABLE', None, ()),
 SchemaField('oral', 'STRING', 'NULLABLE', None, ()),
 SchemaField('topical', 'STRING', 'NULLABLE', None, ()),
 SchemaField('parenteral', 'STRING', 'NULLABLE', None, ()),
 SchemaField('black_box_warning', 'STRING', 'NULLABLE', None, ()),
 SchemaField('applicant_full_name', 'STRING', 'NULLABLE', None, ()),
 SchemaField('innovator_company', 'STRING', 'NULLABLE', None, ()),
 SchemaField('product_id', 'STRING', 'NULLABLE', None, ()),
 SchemaField('nda_type', 'STRING', 'NULLABLE', None, ())]

The schema is a necessary input for one of the more useful commands in BQ: [list_rows](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html#google.cloud.bigquery.client.Client.list_rows). `List_rows` returns a slice of a dataset without scanning any other section of the table. If you've ever written a BQ query that included a `limit` clause, you probably actually wanted `list_rows` instead.

I'd like to see a subset of the columns, but the `selected_fields` parameter requires a schema object as an input. We'll need to build a subset of the schema first to pass for that parameter.

In [ ]:
schema_subset = [col for col in products.schema if col.name in ('product_id',
                                                                'trade_name',
                                                                'applicant_full_name')]
                                                                
results = [x for x in client.list_rows(products, start_index=5000, selected_fields=schema_subset, max_results=10)]

In [ ]:
for i in results:
  print(dict(i)) #print as dicts for easier viewing

{'trade_name': 'FEXOFENADINE HYDROCHLORIDE ALLERGY', 'applicant_full_name': 'DR REDDYS LABORATORIES LTD', 'product_id': 'PRODUCT_076502_006'}
{'trade_name': 'FEXOFENADINE HYDROCHLORIDE HIVES', 'applicant_full_name': 'DR REDDYS LABORATORIES LTD', 'product_id': 'PRODUCT_076502_007'}
{'trade_name': 'FEXOFENADINE HYDROCHLORIDE ALLERGY', 'applicant_full_name': 'DR REDDYS LABORATORIES LTD', 'product_id': 'PRODUCT_076502_008'}
{'trade_name': 'FEXOFENADINE HYDROCHLORIDE HIVES', 'applicant_full_name': 'DR REDDYS LABORATORIES LTD', 'product_id': 'PRODUCT_076502_009'}
{'trade_name': 'METRONIDAZOLE', 'applicant_full_name': 'ABLE LABORATORIES INC', 'product_id': 'PRODUCT_076519_001'}
{'trade_name': 'METRONIDAZOLE', 'applicant_full_name': 'ABLE LABORATORIES INC', 'product_id': 'PRODUCT_076519_002'}
{'trade_name': 'TIZANIDINE HYDROCHLORIDE', 'applicant_full_name': 'APOTEX INC', 'product_id': 'PRODUCT_076533_001'}
{'trade_name': 'TIZANIDINE HYDROCHLORIDE', 'applicant_full_name': 'APOTEX INC', 'product

Suppose we wanted to check what resources we would have consumed by doing a full table scan instead of using list_rows. Looks like the num_bytes method should help us there. (Knowing your query size may be important due to the 5TB monthly quota.)

In [ ]:
def estimate_gigabytes_scanned(query, bq_client):
    # see https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
    my_job_config = bigquery.job.QueryJobConfig()
    my_job_config.dry_run = True
    my_job = bq_client.query(query, job_config=my_job_config)
    BYTES_PER_GB = 2**30
    return my_job.total_bytes_processed / BYTES_PER_GB

In [ ]:
estimate_gigabytes_scanned("SELECT trade_name FROM `patents-public-data.ebi_chembl.products_29`", client)

0.0007636584341526031

In [ ]:
estimate_gigabytes_scanned("SELECT * FROM `patents-public-data.ebi_chembl.products_29`", client)

0.005070987157523632

##Running queries

[View tutorial in BigQuery API Docs](https://googleapis.dev/python/bigquery/latest/usage/queries.html)

In [ ]:
query = """
    SELECT trade_name, applicant_full_name
    FROM `patents-public-data.ebi_chembl.products_29`
    LIMIT 20
"""
query_job = client.query(query)  # Make an API request.

In [ ]:
print("The query data:")
for row in query_job:
    # Row values can be accessed by field name or index.
    print("   trade name: {}, applicant: {}".format(row[0], row[1]))

The query data:
   trade name: XENON XE 133, applicant: GE HEALTHCARE
   trade name: XENON XE 133, applicant: LANTHEUS MEDICAL IMAGING INC
   trade name: XENON XE 133, applicant: LANTHEUS MEDICAL IMAGING INC
   trade name: XENON XE 133, applicant: GENERAL ELECTRIC CO
   trade name: XENON XE 133, applicant: GENERAL ELECTRIC CO
   trade name: XENON XE 133-V.S.S., applicant: GE HEALTHCARE
   trade name: XENON XE 133, applicant: GE HEALTHCARE
   trade name: XENON XE 133, applicant: GE HEALTHCARE
   trade name: MPI KRYPTON 81M GENERATOR, applicant: GE HEALTHCARE
   trade name: XENON XE 133, applicant: CURIUM US LLC
   trade name: XENON XE 133, applicant: CURIUM US LLC
   trade name: XENON XE 127, applicant: MALLINCKRODT MEDICAL INC
   trade name: XENON XE 127, applicant: MALLINCKRODT MEDICAL INC
   trade name: GENOSYL, applicant: VERO BIOTECH
   trade name: NOXIVENT, applicant: PRAXAIR DISTRIBUTION INC
   trade name: NOXIVENT, applicant: PRAXAIR DISTRIBUTION INC
   trade name: DECADERM, app